In [30]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
from tokenizers.trainers import BpeTrainer
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace

In [2]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [3]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [4]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [5]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [31]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [32]:
len(Ytrain), len(Ytest)

(1615, 539)

In [33]:
train_text[:5]

['the unembodied essence and no more',
 'they have not left me as my hopes have since',
 'up attic mother',
 'their solemn peals',
 'of the baubles that it may']

In [34]:
Ytrain[:5]

[0, 0, 1, 1, 0]

In [35]:
idx = 1
word2idx = {'<unk>': 0}

In [36]:
bpe_tokenizer = Tokenizer(BPE())
bpe_tokenizer.pre_tokenizer = Whitespace()
bpe_trainer = BpeTrainer(vocab_size=1000)

bpe_tokenizer.train_from_iterator(train_text, bpe_trainer)

In [37]:
# populate word2idx
for text in train_text:
    tokens = bpe_tokenizer.encode(text).tokens
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [12]:
word2idx

{'<unk>': 0,
 'reply': 1,
 'of': 2,
 'hers': 3,
 'to': 4,
 'our': 5,
 'intelligence': 6,
 'son': 7,
 'knows': 8,
 'the': 9,
 'story': 10,
 'but': 11,
 'twas': 12,
 'not': 13,
 'for': 14,
 'him': 15,
 'i': 16,
 'thought': 17,
 'growing': 18,
 'things': 19,
 'would': 20,
 'do': 21,
 'good': 22,
 'up': 23,
 'fanes': 24,
 'babylonlike': 25,
 'walls': 26,
 'lupine': 27,
 'living': 28,
 'on': 29,
 'sand': 30,
 'and': 31,
 'drouth': 32,
 'you': 33,
 'seem': 34,
 'so': 35,
 'partial': 36,
 'greatgrandmother': 37,
 'conquered': 38,
 'her': 39,
 'scruples': 40,
 'gloom': 41,
 'nor': 42,
 'ghoulhaunted': 43,
 'woodland': 44,
 'weir': 45,
 'thats': 46,
 'why': 47,
 'cant': 48,
 'decently': 49,
 'refuse': 50,
 'that': 51,
 'they': 52,
 'could': 53,
 'hear': 54,
 'tell': 55,
 'was': 56,
 'scarified': 57,
 'when': 58,
 'shes': 59,
 'well': 60,
 'off': 61,
 'is': 62,
 'it': 63,
 'neighbours': 64,
 'in': 65,
 'one': 66,
 'himself': 67,
 'perhaps': 68,
 'im': 69,
 'going': 70,
 'them': 71,
 'something':

In [38]:
len(word2idx)

969

In [40]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = bpe_tokenizer.encode(text).tokens
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = bpe_tokenizer.encode(text).tokens
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [41]:
train_text_int[100:105]

[[131, 229, 436, 227, 437, 74, 33, 162, 59, 237],
 [9, 84, 307, 131, 129, 85, 1, 262, 361],
 [143, 238, 394, 106, 438, 228, 31, 164, 439, 109, 3, 238, 227, 55],
 [440, 191, 441, 108, 410, 164, 442, 33, 216],
 [443, 79, 154, 444, 16, 37, 229, 276, 106, 118]]

In [42]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [43]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [44]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [45]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [46]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.34365325077399383, 0.6563467492260062)

In [47]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [48]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [49]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9969040247678018


In [50]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8385899814471243


In [51]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [52]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 550,    5],
       [   0, 1060]])

In [53]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 95,  68],
       [ 19, 357]])

In [54]:
f1_score(Ytrain, Ptrain)

0.9976470588235294

In [55]:
f1_score(Ytest, Ptest)

0.8913857677902621